In [1]:
import pandas as pd
import numpy as np
import joblib
from collections import Counter
from joblib import load

# --------------------
# 1. Charger les modèles
# --------------------
encoder = load(r"C:\Users\fgrol\Documents\stages\models\label_encoder.pkl")
knn_model = joblib.load(r"C:\Users\fgrol\Documents\stages\models\knn_model.pkl")
rf_model = joblib.load(r"C:\Users\fgrol\Documents\stages\models\rf_model.pkl")
catboost_model = joblib.load(r"C:\Users\fgrol\Documents\stages\models\catboost_model.pkl")
svm_model = joblib.load(r"C:\Users\fgrol\Documents\stages\models\svm_model.pkl")

# --------------------
# 2. Colonnes features
# --------------------
colonnes_features = [
    'x', 'y', 'altitude', 'hauteur', 'vitesse',
    'vitesse_calculee', 'delta_t', 'vz',
    'distance', 'temps_total', 'distance_totale',
    'longitude_depart'
]

# --------------------
# 3. Charger fichier CSV
# --------------------
fichier = r"C:\Users\fgrol\Documents\stages\csv_test.csv"
df = pd.read_csv(fichier, sep=',', quotechar='"', engine='python', nrows=10000)

# --------------------
# 4. Nettoyer DataFrame
# --------------------
X = df[colonnes_features].apply(pd.to_numeric, errors='coerce')
X = X.replace([np.inf, -np.inf], np.nan)
print("NaNs par colonne avant suppression :")
print(X.isna().sum())
X = X.dropna()
print(f"Lignes après nettoyage: {len(X)}")

# --------------------
# 5. Prédictions
# --------------------
pred_knn = encoder.inverse_transform(knn_model.predict(X))
pred_rf = encoder.inverse_transform(rf_model.predict(X))
pred_cb = encoder.inverse_transform(catboost_model.predict(X))
pred_svm = encoder.inverse_transform(svm_model.predict(X))

# --------------------
# 6. Ajouter prédictions au DataFrame
# --------------------
df_predictions = df.loc[X.index].copy()
df_predictions['KNN'] = pred_knn
df_predictions['RandomForest'] = pred_rf
df_predictions['CatBoost'] = pred_cb
df_predictions['SVM'] = pred_svm

# --------------------
# 7. Calcul vote majoritaire
# --------------------
def vote_majoritaire(row):
    votes = [row['KNN'], row['RandomForest'], row['CatBoost'], row['SVM']]
    vote_counts = Counter(votes)
    return vote_counts.most_common(1)[0][0]

df_predictions['majorité'] = df_predictions.apply(vote_majoritaire, axis=1)

# --------------------
# 8. Calcul % de justesse
# --------------------
def accuracy(y_true, y_pred):
    return (y_true == y_pred).mean() * 100

from IPython.display import Markdown

accuracy_dict = {
    'KNN': accuracy(df_predictions['modele_standardise'], df_predictions['KNN']),
    'Random Forest': accuracy(df_predictions['modele_standardise'], df_predictions['RandomForest']),
    'CatBoost': accuracy(df_predictions['modele_standardise'], df_predictions['CatBoost']),
    'SVM': accuracy(df_predictions['modele_standardise'], df_predictions['SVM']),
    'Vote majoritaire': accuracy(df_predictions['modele_standardise'], df_predictions['majorité']),
}

df_scores = pd.DataFrame.from_dict(accuracy_dict, orient='index', columns=['Taux de réussite (%)'])
df_scores.index.name = 'Modèle'
df_scores = df_scores.round(2)

display(Markdown("### 🎯 **Taux de justesse par modèle**"))
display(df_scores.style.set_properties(**{'text-align': 'center'}))

# --------------------
# 8bis. Ajout colonne Modèle depuis drone_id
# --------------------

# Dictionnaire code drone → modèle
drones_dict = {
    "1581E07J": "Phantom 4 (MainControl)",
    "1581E08R": "Mavic Pro",
    "1581E09Y": "Inspire 2",
    "1581E0K1": "Mavic Air",
    "1581E0M6": "Mavic 2 Zoom",
    "1581E0UY": "Phantom 4 RTK (FlightControl)",
    "1581E0V2": "Phantom 4 RTK",
    "1581E11V": "Phantom 4 Pro",
    "1581E163": "Mavic 2 Pro",
    "1581E1DA": "Matrice 210 RTK",
    "1581E1SC": "Mavic Mini",
    "1581E1SD": "Mavic Mini (CoreBoard)",
    "1581E1WG": "Mavic Mini",
    "1581E1WN": "Mavic Air 2",
    "1581E1ZN": "Matrice 300 RTK",
    "1581E276": "Mavic 2 Enterprise",
    "1581E298": "Mavic 2 Enterprise Dual",
    "1581E37Q": "DJI FPV",
    "1581E3N3": "Mavic Air 2",
    "1581E3NZ": "Mini 2",
    "1581E4DT": "Mini SE (CoreBoard)",
    "1581E3YT": "Air 2S",
    "1581E4GC": "Mavic 2 Enterprise Advanced",
    "1581E5FS": "Mini 2 (CoreBoard)",
    "1581E687": "Mini 3 (CoreBoard)",
    "1581E6C5": "Mini 2 SE (CoreBoard)",
    "1581E8JE": "Mini 4K (CoreBoard)",
    "1581F3YT": "Air 2S",
    "1581F45T": "Mavic 3",
    "1581F4QW": "Avata",
    "1581F4QZ": "Mavic 3 Cine",
    "1581F4XF": "Mini 3 Pro",
    "1581F4Z4": "Inspire 3",
    "1581F5BK": "Matrice 30T",
    "1581F5FH": "Mavic 3E",
    "1581F5FJ": "Mavic 3T",
    "1581F5FK": "Mavic 3M",
    "1581F5YH": "Mini 3",
    "1581F62H": "Matrice 30T",
    "1581F67P": "Mavic 3 Classic",
    "1581F67Q": "Mavic 3 Pro",
    "1581F6CD": "Mini 2 SE",
    "1581F6GK": "Matrice 350 RTK",
    "1581F6L5": "Mini 3 Pro",
    "1581F6MK": "Mavic 3 Pro Cine",
    "1581F6N8": "Air 3",
    "1581F6Q8": "Matrice 3TD",
    "1581F6W8": "Avata 2",
    "1581F6Z9": "Mini 4 Pro",
    "1581F7FV": "Matrice 4E",
    "1581F7V2": "Flip",
    "1581F87L": "Neo",
    "1581F895": "Air 3S",
    "1581F8LQ": "Mavic 4 Pro",
    "1581F8PJ": "Mini 4K",
    "1581F986": "Mavic 4 Pro 512GB"
}

# Extraction du préfixe
df_predictions['prefixe_id'] = df_predictions['drone_id'].str[:8]

# Ajout d'une colonne "Modèle"
df_predictions['Modèle'] = df_predictions['prefixe_id'].map(drones_dict).fillna('Inconnu')


# --------------------
# 9. Fusion par drone uniquement
# --------------------

# Fonction pour prédiction dominante par drone
def mode_majoritaire(series):
    return series.mode().iloc[0] if not series.mode().empty else 'Inconnu'

grouped_drone = df_predictions.groupby('drone_id').agg({
    'KNN': mode_majoritaire,
    'RandomForest': mode_majoritaire,
    'CatBoost': mode_majoritaire,
    'modele_standardise': mode_majoritaire,
    'majorité': mode_majoritaire
}).reset_index()

# --------------------
# 10. Ajouter infos supplémentaires
# --------------------

# Prédiction majorité par drone
pred_maj_par_drone = df_predictions.groupby('drone_id')['majorité'].agg(
    lambda x: x.mode().iloc[0] if not x.mode().empty else 'Inconnu'
)

# Ajout dans le DataFrame
grouped_drone['Prédiction majorité (drone)'] = grouped_drone['drone_id'].map(pred_maj_par_drone)

# Ajout du modèle en rognant drone_id
grouped_drone['Modèle'] = grouped_drone['drone_id'].str[:8].map(drones_dict).fillna('Inconnu')

# Renommer pour correspondre à l'affichage Voilà
df_voila = grouped_drone.rename(columns={
    'drone_id': 'Drone ID',
    'modele_standardise': 'Modèle réel',
    'KNN': 'KNN',
    'RandomForest': 'Random Forest',
    'CatBoost': 'CatBoost',
    'majorité': 'Prédiction majorité'
})

# Réordonner les colonnes
df_voila = df_voila[
    ['Drone ID', 'Modèle réel', 'KNN', 'Random Forest', 'CatBoost', 'Prédiction majorité (drone)', 'Modèle']
]

# --------------------
# 11. Affichage Voilà stylisé (sans Correct)
# --------------------
styled_df = df_voila.style \
    .set_properties(**{'text-align': 'center'}) \
    .set_table_styles([dict(selector='th', props=[('text-align', 'center')])])

display(styled_df)


import ipywidgets as widgets
from IPython.display import FileLink, display

# Créer un lien de téléchargement
chemin_fichier = r"C:\Users\fgrol\Documents\stages\algo\csv_predictions_output.csv"
bouton_telechargement = widgets.Button(description="📥 Télécharger le fichier CSV", button_style='success')

def telecharger_fichier(b):
    display(FileLink(chemin_fichier, result_html_prefix="Cliquez ici pour télécharger : "))

bouton_telechargement.on_click(telecharger_fichier)

display(bouton_telechargement)


NaNs par colonne avant suppression :
x                     0
y                     0
altitude              0
hauteur              77
vitesse               0
vitesse_calculee      0
delta_t              27
vz                  230
distance              0
temps_total           0
distance_totale       0
longitude_depart     39
dtype: int64
Lignes après nettoyage: 9631


c:\Users\fgrol\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\preprocessing\_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\fgrol\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


### 🎯 **Taux de justesse par modèle**

,Taux de réussite (%)
Modèle,
KNN,97.190000
Random Forest,99.700000
CatBoost,99.180000
SVM,0.000000
Vote majoritaire,99.330000


,Drone ID,Modèle réel,KNN,Random Forest,CatBoost,Prédiction majorité (drone),Modèle
0,000MCT000000000000409151886510,Extra 330S,Mavic 3 Pro,Mavic 3,Matrice 30T,Mavic 3 Pro,Inconnu
1,1581E0M6CG9QR0A0EFB,Mavic 2 Zoom,Mavic 3 Pro,Mavic 2 Zoom,Mavic 2 Zoom,Mavic 2 Zoom,Mavic 2 Zoom
2,1581E0UYKH8J00200HK,Phantom 4 RTK,Phantom 4 RTK,Phantom 4 RTK,Phantom 4 RTK,Phantom 4 RTK,Phantom 4 RTK (FlightControl)
3,1581E11VKF4D00201NS,Phantom 4 Pro,Avata,Phantom 4 Pro,Matrice 350 RTK,Avata,Phantom 4 Pro
4,1581E11VKF6600202A5,Phantom 4 Pro,Phantom 4 Pro,Phantom 4 Pro,Phantom 4 Pro,Phantom 4 Pro,Phantom 4 Pro
5,1581E11VKH3X00200ZZ,Phantom 4 Pro,Phantom 4 Pro,Phantom 4 Pro,Phantom 4 Pro,Phantom 4 Pro,Phantom 4 Pro
6,1581E1633GBE0H1Y025,Mavic Pro,Mavic Pro,Mavic Pro,Mavic Pro,Mavic Pro,Mavic 2 Pro
7,1581E1633H4W0HSK004,Mavic Pro,Mavic Pro,Mavic Pro,Mavic Pro,Mavic Pro,Mavic 2 Pro
8,1581E1633HAF0H1X089,Mavic Pro,Mavic Pro,Mavic Pro,Mavic Pro,Mavic Pro,Mavic 2 Pro
9,1581E1633JC90H1Z041,Mavic Pro,Mavic 3 Pro,Mavic Pro,Mavic 3 Classic,Mavic 3 Classic,Mavic 2 Pro


Button(button_style='success', description='📥 Télécharger le fichier CSV', style=ButtonStyle())